In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = None
pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
import warnings
import regex as re
warnings.filterwarnings('ignore')
from decimal import *
TWOPLACES = Decimal(10) ** -2
from natsort import natsorted
import ast
import time
today = time.strftime("%Y_%m_%d")
from enrich_dimensions.rounds import rounds, rounding_inch_feet,rounding_lbs,rounding_w,rounding_oz, rounding_lb,rounding_gal, re_extract, curate, round_string_float, clean_data,reg_ex,rounding_period_after_unit 
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

# parameters
customer_id = '5'
formatted_attribute = 'Diameter'
customer_name='%bedbathandbeyond%'
buckets = """Office Chairs"""

strategy_version_input=767
attribute_id_input=2730

attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'


df = query_from_file(file_name='./query/custom_fields.sql', params=params)
print(len(df))
print(today)

2513
2023_04_25


In [2]:
df.drop_duplicates(subset='external_id', inplace=True)
df=df[(df['external_id'].astype(str)!='200045713')&(df['external_id'].astype(str)!='200045715')&(df['external_id'].astype(str)!='200045714')]
import collections
print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])

[]


In [3]:
regex_pattern_diameter=r'(?i)(\d+.?(?:cm|mm).?Diam?e?t?e?r?\b)|()'
df['mm'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_diameter, x))
df=df[df['mm'].astype(str)=='[]']
del df['mm']
print(len(df))

2501


# Good Stuff

In [4]:
regex_pattern_diameter=r'(?i)(\d+\.\d+.?.?.?Diam?e?t?e?r?\b)|(\d+.?\d+\/\d+.?.?.?Diam?e?t?e?r?\b)|((?<!\.)(?<!\/)\d+(?!\.\d).?.?.?Diam?e?t?e?r?\b)|((?<!\.)(?<!\/)\d+(?!\.).?.?.?Diam?e?t?e?r?\b)|()'
regex_pattern_diameter=r'''(?i)((?<!\/)(?<!\d)(?<!\,)(?<!\.)(?:\d+\.)?\d+(?:\\|\-|\\\\)?(?:"|'|inc?h?e?s?|fe?e?o?o?t)[^\w]{0,3}Diam?e?t?e?r?\b)|((?<!\/)(?<!\d)(?<!\,)(?<!\.)(?:\d+\.)?\d+(?:\\|\-|\\\\)?(?:"|'|inc?h?e?s?|fe?e?o?o?t)[^\w]{0,3}Diam?e?t?e?r?\b)|((?<!\/)(?<!\d)(?<!\,)(?<!\.)(?:\d+\.)?\d+(?:\\|\-|\\\\)?(?:"|'|inc?h?e?s?|fe?e?o?o?t)[^\w]{0,3}Diam?e?t?e?r?\b)|((?<!\/)(?<!\d)(?<!\,)(?<!\.)(?:\d+\.)?\d+(?:\\|\-|\\\\)?(?:"|'|inc?h?e?s?|fe?e?o?o?t)[^\w]{0,3}Diam?e?t?e?r?\b)|()'''

df['matches_diameter'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_diameter, x))
regex_pattern_ft=r"\d'.?.?.?[LWHD]|\d\'.?diameter|(\bround\b)|(circ)|(lamp)"
df['matches_ft'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_ft, x))
regex_pattern_blank=r'\d'
df['new_matches'] = df['matches_diameter'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
regex_pattern_blank=r'(?i)(\d+.?\d*\/?\d*.?I?n?c?h?"?.?x.?\d+.?\d*\/?\d*.?I?n?c?h?"?)|(\d+.?ft)|(\d+.?foot)|(\d+.?feet)|(\d+.?inch)|(\d+.?")|(\d+")|()'
df['blank_matches'] = df['product_name'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
df['blank_matches_long'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))

In [5]:
# x=df[df['matches_diameter'].astype(str)!='[]']
# print(len(x))

# x['Q:diameter']=x['matches_diameter'].apply(lambda x: re.sub(r'(?i)(diam?e?t?e?r?)|(\\)','',str(x))).apply(lambda x: re.sub(r'''(?i)(\s?''\s?)|((?<=\d)"\s?)|(\s?in\s?)''',' in',str(x))).apply(lambda x: re.sub(r'''(?i)((?<=\d)')|((?<=\d)ft)''',' ft',str(x)))               
# x['Q:diameter'].explode().value_counts()
# print(len(x))
# x

In [6]:
height_new=df[(df['matches_diameter'].astype(str)!='[]')&(df['matches_ft'].astype(str)=='[]')]
height_new['matches_diameter']=height_new['matches_diameter'].apply(lambda x: natsorted(x))
height_new['matches_diameter']=height_new['matches_diameter'].apply(lambda x: re.sub(r"\\?'.?diam?e?t?e?r?"," ft",str(x))).apply(lambda x: re.sub(r'(?i)\\?\".?diam?e?t?e?r?|\\?\”?.?diam?e?t?e?r?|\\\" H',' in',str(x))).apply(lambda x: re.sub(r'\\" H|"H|\"xa0H|xa0?H?|"|®','',str(x))).apply(lambda x: re.sub(r"\[|\]|'|\\|’’",'',str(x))).apply(lambda x: re.sub(r"-",' ',str(x)))
print(len(height_new))

38


In [7]:
curate(height_new, 'matches_diameter', formatted_attribute)
print(len(height_new))
rounding_inch_feet(height_new, curation_col,'a-eghj-mo-su-z')
height_news=height_new[height_new['rounding'].astype(str)=='[]']
print(len(height_news))
matchesdiam=height_news[['external_id',curation_col]]
print('Number of SKUs for Diameter: '+str(len(matchesdiam)))
rounding_inch_feet(height_news, curation_col,'a-eghj-mo-su-z')

Repeated External IDs: []
38
32
Number of SKUs for Diameter: 32


Empty DataFrame
Columns: [buckets, bucket_id, product_id, external_id, product_name, long_desc, url, image_url, custom_fields, value, matches_diameter, matches_ft, new_matches, blank_matches, blank_matches_long, ones, twos, threes, fours, fives, Q:diameter, rounding]
Index: []

# NA

In [8]:
# height_new[height_new['external_id'].astype(str)=='200103897']

In [9]:
df_na=df[(df['matches_diameter'].astype(str)=='[]')]
print(len(df_na))
df_na[curation_col]='n/a'
matchesna=df_na[['external_id',curation_col]]

2368


In [10]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchesna))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchesdiam))

matchesna
matchesdiam


In [11]:
rounding_inch_feet(height_new, curation_col,'a-eghj-mo-su-z')

buckets  bucket_id  product_id external_id  \
753   Pails, Buckets & Accessories       9749    10860336   201058733   
1380           Bracelets & Anklets       7630     2401147   201215335   
1445           Bracelets & Anklets       7630    10711974   201185131   
1446           Bracelets & Anklets       7630    10711975   201184859   
1447           Bracelets & Anklets       7630    10711976   201189162   
1560           Bracelets & Anklets       7630    11151983   201188041   

                                                                                          product_name  \
753                                      Homz Plastic Rope-Handled Storage Tub 18 Gallon Set of 2 Pink   
1380                    GoodMerchan - Women's Motivational Cuff Bracelet - Empower yourself with style   
1445                    GoodMerchan - Women's Motivational Cuff Bracelet - Empower yourself with style   
1446                    GoodMerchan - Women's Motivational Cuff Bracelet - Empower yourself with style   
1447                    GoodMerchan - Women's Motivational Cuff Bracelet - Empower yourself with style   
1560  GoodMerchan - Men's Open Cuff Bracelet Set - Gold, 3 Pieces - Stackable Summer Bracelets Jewelry   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 long_desc  \
753                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               The Homz 18 gallon utility tub is great for organizing the garage, as a laundry basket, a cooler for outdoor parties or even as a toy bin for children. It is made from durable plastic that withstands ongoing use. Great for indoor or outdoor use. The open design allows for easy access storage. They nest inside one another when not in use for efficient storing. The strong nylon rope handles make for easy carrying. The Exterior Dimensions are 21.5\". diameter x 16.5\" high. Interior Dimensions 16.25in. x 16.25in. x 16.4in. Color Pink tub with white rope handles. Homz Plastic Utility Tub with Rope Handles, 18 Gallon, Pink, Set of 2 18 gallon capacity Rugged plastic material with rope handles withstands indoor and outdo

In [12]:
print(len(df))
print(len(matchesna)+len(matchesdiam))

2501
2400


In [13]:
# height_new[height_new['matches_diameter'].astype(str)!='[]']

In [14]:
# height_new

In [15]:
stop

NameError: name 'stop' is not defined

# send to the folder for upload

In [16]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-{today}-matches.csv',index=False) 
looks_good('Bed Bath & Beyond', attribute,matchesna,today)   

In [17]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
def looks_good(customer, attribute,matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-{today}-matches.csv',index=False) 
looks_good('Bed Bath & Beyond', attribute,matchesdiam,today)  